In [1]:
import os
dir_path = "/home/wouter/onesnake"
os.chdir(dir_path)
print("loading py files from:", os.getcwd())
import shutil
import logging
import matplotlib.pyplot as plt
import time
import subprocess
import numpy as np
from simulation import Simulation
from snakesimulation import snakeSimulation
from forcedsimulation import forcedSimulation
from extsimulation import extSimulation
from funcs import plot_paths, overlay_paths, plot_2Dpaths
from vvengine import VelocityVerletEngine
from funcs import get_introspective_swap_matrix
from funcs import select_submatrix, permanent_prob, fast_glynn_perm
from funcs import fastpermanent_repeat_prob
import permanent

%matplotlib qt
%load_ext autoreload


work_dir = "./sshuffleboard_retis"

# if os.path.exists(work_dir):
#     shutil.rmtree(work_dir)
#     print('removed old work_dir')
# if not os.path.exists(work_dir):
#     os.mkdir(work_dir)
#     # copy the file "permanent" to work_dir
#     shutil.copy("permanent", work_dir)
#     shutil.copy('retis.rst', work_dir)
#     # copy the file "maze.png" to work_dir
#     shutil.copy('maze.png', work_dir)
#     #shutil.copy("retis_initpath_phs.npy", work_dir)
#     shutil.copy("retis_shuffle_initpath.npy", work_dir)
# # # copy all py files of cwd to test
# # for file in os.listdir(os.getcwd()):
# #     if file.endswith(".py"):
# #         shutil.copy(file, work_dir)
os.chdir(work_dir)
print("Copied files and put cwd to:", os.getcwd())


loading py files from: /home/wouter/onesnake


Copied files and put cwd to: /home/wouter/onesnake/sshuffleboard_retis


In [2]:
# %autoreload
# from potential import RectangularGridWithBarrierPotential
# Lx = 15
# Ly = 5
# k = 2
# M = (7.5, 2.5)
# r = -3*Lx/Ly 
# A = 2
# L = 2 
# testpot = RectangularGridWithBarrierPotential(Ly, Lx, k, M, r, A, L)
# x = np.linspace(-Lx*.1, Lx*1.1, 200)
# y = np.linspace(-Ly*.1, Ly*1.1, 200)
# vx = np.zeros_like(x)
# vy = np.zeros_like(y)
# X, Y = np.meshgrid(x, y)
# Z = np.zeros_like(X)
# for i in range(len(x)):
#     for j in range(len(y)):
#         Z[i, j] = testpot.potential_and_force([np.array([x[i], y[j]]), np.array([0, 0])])[0]
# fig = plt.figure()
# ax = fig.add_subplot(111)
# f = ax.contourf(X, Y, Z, 200)
# # show contour lines at 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
# plt.contour(X, Y, Z, levels=[-5, 0, 9])
# # x and y scale are equal 
# #plt.axis('equal')
# # equal aspect ratio
# ax.set_aspect('equal')
# fig.colorbar(f)
# fig.tight_layout()
# fig.show()

In [2]:
intfs = np.linspace(0.5,8.5,11)*.1

# Set-up the simulation keys for RETIS
retisset = {"interfaces" : intfs.tolist(),
            "simtype" : "retis", 
            "method" : "load",
            "max_len" : 10000000,
            "dt": 0.002,
            "temperature": .05,
            "friction": 25.,
            "max_cycles": 250000,
            "p_shoot": 0.75,
            "include_stateB": False,
            'prime_both_starts': False,
            'snake_Lmax': 0,
            'max_paths': 1,
            'Nl': 1,
             'dim' : 2,
             'mass' : .1,
}
# or for REPPTIS
repptisset = {"interfaces" : intfs,
             "simtype" : "repptis", 
             "method" : "load",
             "max_len" : 1000000,
             "dt": 0.002,
             "temperature": .05,
             "friction": 25.,
             "max_cycles": 200000,
             "p_shoot": 1.,
             "include_stateB": True,
             'prime_both_starts': True,
             'snake_Lmax': 0,
             'max_paths': 2,
             'sL' : 9,
             'Nl': 1,
             'Next' : 3,
             'yoot' : False, # to set choice = 0 and W = 1.
             'endpoints_only' : False,
             'flipboth': True,
             'binomial': True,
             'random_shoots': True,
             'invert_W' : False,
             'dim' : 2,
             'mass' : .1,
}

%autoreload
logger = logging.getLogger()
file_handler = logging.FileHandler("logging.log")
formatter = logging.Formatter('[%(levelname)s] %(name)s %(funcName)s %(lineno)d: %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.setLevel(logging.INFO)
logger.info("Hello world!")
logger.propagate = False
sim = Simulation(retisset)
xpot, ypot, vpot = sim.ensembles[0].engine.potential.get_potential_plot()

# sim = extSimulation(repptisset)
# xpot, ypot, vpot = sim.ensembles[0][0].engine.potential.get_potential_plot()


# set loglevel to warning for matplotlib
logging.getLogger("matplotlib").setLevel(logging.WARNING)


In [3]:
# %autoreload
sim.run()

KeyboardInterrupt: 

In [6]:
sim.max_cycles = 50000
sim.run()

In [5]:
%autoreload
from funcs import plot_2Dpaths
fig, ax = plt.subplots()
for intf in sim.intfs:
    ax.axhline(intf, color='black', ls='--')
f = ax.contourf(xpot, ypot, vpot, levels=[-0.2,-.1,0,0.1,0.2,0.3,1,2,5])
fig.colorbar(f)
plot_2Dpaths([ens.last_path for ens in sim.ensembles], ax=ax, intfs=sim.intfs, ls="-")
fig.show()


# fig, ax = plt.subplots()
# for ens in sim.ensembles:
#         plot_2Dpaths([ens.last_path], intfs=intfs, ax=ax)
# fig.show()

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [15]:
%autoreload
phasepoint = (np.array([0.1,0.1]), np.array([0,0]))
path = []
eng = sim.ensembles[0].engine
n = 0
forces = []
while n < 1000:
    ph,force = eng.step(phasepoint)
    path.append(ph)
    forces.append(force)
    phasepoint = ph 
    n += 1
savepath = path.copy()
path = np.array(path)
# average force and std dev
forces = np.array(forces)
favg = np.mean(forces, axis=0)
fstd = np.std(forces, axis=0)
print("average force:", favg)
print("std dev force:", fstd)


average force: [-1.90620336  1.15825148]
std dev force: [41.53502249 26.18070085]


In [16]:
fig, ax = plt.subplots()
for intf in sim.intfs:
    ax.axhline(intf, color='black', ls='--')
ax.plot(path[:, 0, 0], path[:, 0, 1], ls='-', marker='x',ms=1,lw=.1)
fig.show()

In [17]:
newpathsave = []
# we go through the phasepoints of savepath, if it is < intf[0] or > intf[-1] we don't add it to newpathsave
for ph in savepath:
    if ph[0][1] < intfs[0] or ph[0][1] > intfs[-1]:
        continue
    newpathsave.append(ph)
# now add one point that is < intfs[0] to newpathsave (at the beginning)
while True:
    for ph in savepath:
        if ph[0][1] < intfs[0]:
            newpathsave.insert(0, ph)
            break
    break 
while True:
#and one point that is > intfs[-1] to newpathsave (at the end)
    for ph in savepath[::-1]:
        if ph[0][1] > intfs[-1]:
            newpathsave.append(ph)
            break
    break
newpathsave = np.array(newpathsave)
np.save("retis_shuffle_initpath.npy", newpathsave)

In [2]:
# RESTART PROCEDURE
import pickle as pkl
import os
with open('sim.pkl', 'rb') as f:
    sim = pkl.load(f)
import logging
logger = logging.getLogger()
file_handler = logging.FileHandler("logging.log")
formatter = logging.Formatter('[%(levelname)s] %(name)s %(funcName)s %(lineno)d: %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.setLevel(logging.INFO)
logger.info("Hello world!")
logger.propagate = False
logging.getLogger("matplotlib").setLevel(logging.WARNING)


In [3]:
sim.max_cycles=50000

sim.run()

KeyboardInterrupt: 

In [4]:
sim.max_cycles = 140000

In [8]:
sim.cycle

200000

In [5]:
import pickle as pkl
import os
with open('./shuffleboard_reppextis3/sim.pkl', 'rb') as f:
    sim = pkl.load(f)

FileNotFoundError: [Errno 2] No such file or directory: './shuffleboard_reppextis3/sim.pkl'

In [6]:
print("haha")

haha


In [4]:
fig, ax = plt.subplots()
l, e = 0, 3
for intf in sim.intfs:
    ax.axhline(intf, c="black", linestyle="--")
plot_paths([connec[1] for connec in sim.ensembles[l][e].last_path.connections], ax = ax, start_ids="staggered")
plot_paths([sim.ensembles[l][e].last_path], ax = ax, c="purple")

'/home/wouter/onesnake'